In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
import pandas as pd
from extract import open_driver, close_driver, wait_for_element
from transforms import parse_record, parse_game_date
from selenium.webdriver.common.by import By
from DML import db_connect, close_db_connect
from sql_queries import summary_table_insert

In [4]:
def download(url, driver):
    driver.get(url)
    root_element = wait_for_element(source=driver, search_by=By.ID, target="root", unique_element=True)
    data_table = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="rt-table", unique_element=True)
    
    table_headers = get_table_columns(data_table)
    table_data = get_table_rows(data_table, table_headers)
    
    return table_data

In [5]:
def get_table_columns(source):
    table_headers = wait_for_element(source=source, search_by=By.CLASS_NAME, target="tableHeaderDiv", unique_element=True)
    headers = wait_for_element(source=table_headers, search_by=By.CLASS_NAME, target="rt-th")
    header_names = [header.text for header in headers]
    
    return header_names

In [6]:
def get_table_rows(source, headers):
    table_rows = wait_for_element(source=source, search_by=By.CLASS_NAME, target="rt-tbody", unique_element=True)
    rows = wait_for_element(source=table_rows, search_by=By.CLASS_NAME, target="rt-tr")
    table_data = []
    
    for row in rows:
        cells = wait_for_element(source=row, search_by=By.CLASS_NAME, target="rt-td")
        row_values = [cell.text for cell in cells]
        
        values_map = list(zip(headers, row_values))
        map_dict = parse_record(values_map)
        
        table_data.append(map_dict)
            
    return table_data

In [10]:
def scrape_page(url):
    driver = open_driver()
    table_data = download(url, driver)
    close_driver(driver)
    
    # call batch upload function
    
    return table_data

In [ ]:
# TODO: batch update function

In [11]:
url = "https://www.nhl.com/stats/teams?aggregate=0&reportType=game&dateFrom=2021-10-12&dateTo=2022-04-26&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=100"

In [12]:
table_data = scrape_page(url=url)

[{'i': 1, 'opponent': 'STL', 'home_game': False, 'game_date': datetime.datetime(2022, 2, 10, 0, 0), 'GP': 1, 'W': 1, 'L': 0, 'OT': 0, 'P': 2, 'P%': 1.0, 'RW': 1, 'ROW': 1, 'S/O Win': 0, 'GF': 7, 'GA': 4, 'GF/GP': 7.0, 'GA/GP': 4.0, 'PP%': 0.0, 'PK%': 100.0, 'Net PP%': 0.0, 'Net PK%': 100.0, 'Shots/GP': 29.0, 'SA/GP': 31.0, 'FOW%': 55.2}, {'i': 2, 'opponent': 'CAR', 'home_game': True, 'game_date': datetime.datetime(2022, 1, 22, 0, 0), 'GP': 1, 'W': 1, 'L': 0, 'OT': 0, 'P': 2, 'P%': 1.0, 'RW': 1, 'ROW': 1, 'S/O Win': 0, 'GF': 7, 'GA': 4, 'GF/GP': 7.0, 'GA/GP': 4.0, 'PP%': 33.3, 'PK%': 100.0, 'Net PP%': 33.3, 'Net PK%': 100.0, 'Shots/GP': 38.0, 'SA/GP': 32.0, 'FOW%': 40.6}, {'i': 3, 'opponent': 'EDM', 'home_game': True, 'game_date': datetime.datetime(2021, 12, 31, 0, 0), 'GP': 1, 'W': 1, 'L': 0, 'OT': 0, 'P': 2, 'P%': 1.0, 'RW': 0, 'ROW': 1, 'S/O Win': 0, 'GF': 6, 'GA': 5, 'GF/GP': 6.0, 'GA/GP': 5.0, 'PP%': 50.0, 'PK%': 100.0, 'Net PP%': 50.0, 'Net PK%': 100.0, 'Shots/GP': 41.0, 'SA/GP': 